In [ ]:
# import packages

# data processing
import pandas as pd
import numpy as np
from datetime import timedelta, datetime


import re

# data visualization
import plotly.graph_objs as go
from plotly.graph_objs import Bar, Layout
from plotly import offline
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 10)

plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号

# change text color
import colorama
from colorama import Fore, Style

# IPython
from IPython.display import IFrame

from sklearn.feature_selection import mutual_info_classif

%matplotlib inline

## Scoring Metrics

<div class="alert alert-block alert-success"><b>Step 1</b>: 
    

1. normalized gini coefficent
    
</div>

In [ ]:
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = -eval_gini(labels, preds)
    return [('gini', gini_score)]

## Underfitting AND Overfitting

### Underfitting
A statistical model or a machine learning algorithm is said to have underfitting when a model is too simple to capture data complexities. It represents the inability of the model to learn the training data effectively result in poor performance both on the training and testing data. In simple terms, an underfit model’s are inaccurate, especially when applied to new, unseen examples. It mainly happens when we uses very simple model with overly simplified assumptions. To address underfitting problem of the model, we need to use more complex models, with enhanced feature representation, and less regularization.

### Overfitting
A statistical model is said to be overfitted when the model does not make accurate predictions on testing data. When a model gets trained with so much data, it starts learning from the noise and inaccurate data entries in our data set. And when testing with test data results in High variance. Then the model does not categorize the data correctly, because of too many details and noise. The causes of overfitting are the non-parametric and non-linear methods because these types of machine learning algorithms have more freedom in building the model based on the dataset and therefore they can really build unrealistic models. A solution to avoid overfitting is using a linear algorithm if we have linear data or using the parameters like the maximal depth if we are using decision trees. 

**Reasons for Underfitting:**
1.The model is too simple, So it may be not capable to represent the complexities in the data.
2.The input features which is used to train the model is not the adequate representations of underlying factors 3.influencing the target variable.
4.The size of the training dataset used is not enough.
5.Excessive regularization are used to prevent the overfitting, which constraint the model to capture the data well.
6.Features are not scaled.

**Techniques to Reduce Underfitting**
1.Increase model complexity.
2.Increase the number of features, performing feature engineering.
3.Remove noise from the data.
4.Increase the number of epochs or increase the duration of training to get better results.

**Reasons for Overfitting:**
1.High variance and low bias.
2.The model is too complex.
3.The size of the training data.

**Techniques to Reduce Overfitting:**
1.Increase training data.
2.Reduce model complexity.
3.Early stopping during the training phase (have an eye over the loss over the training period as soon as loss begins 4.to increase stop training).
5.Ridge Regularization and Lasso Regularization.
6.Use dropout for neural networks to tackle overfitting.

## Cross validation

Cross-validation is a statistical method used to evaluate the performance of machine learning models. It involves dividing the available data into multiple folds or subsets, using one of these folds as a validation set, and training the model on the remaining folds.

### When to use Cross Validation

Cross-validation is used when data is not sufficient. For example, if the data sample size is less than 10,000, we will use cross-validation to train the optimization selection model. If there are more than 10,000 samples, we usually randomly divide the data into three parts, one is the training set (Training Set), one is the validation set (Validation Set), and the last part is the test set (Test Set). 

We Use the training set to train the model, and the validation set to evaluate the quality of model predictions and select the model and its corresponding parameters. Use the final model on the test set to finally decide which model to use and the corresponding parameters.

### What are the uses of cross-validation?

1.Model selection, also known as hyperparameter selection.
2.Model evaluation, using cross-validation to assess the performance of a model.

### What are the methods of cross-validation?

**The first one is Simple Cross-Validation:**
The term 'simple' is relative to other cross-validation methods. Initially, the sample data is randomly divided into two parts (e.g., 70% training set and 30% test set). Then, the model is trained on the training set and validated on the test set. After that, the samples are shuffled, and new training and test sets are chosen to continue training and checking the model. Finally, a loss function is used to evaluate the best model and parameters.

**The second one is S-Fold Cross-Validation:**
Unlike the first method, S-Fold Cross-Validation randomly divides the sample data into S parts. In each round, S-1 parts are chosen as the training set, and the remaining 1 part is used as the test set. After this round, a new random selection of S-1 parts is used for training the data. After several rounds (fewer than S), a loss function is used to evaluate the best model and parameters.

**The third one is Leave-One-Out Cross-Validation:**
It is a special case of the second method, where S is equal to the number of samples, N. In this case, for N samples, in each round, N-1 samples are used for training, leaving one sample for validating the quality of the model's predictions. This method is mainly used when the sample size is very small, such as in cases where N is less than 50, for moderately sized problems. I generally use Leave-One-Out Cross-Validation in such cases."

    
### Difference between the validation set and the test set:

1.The validation set is used during the training process to assess the model's training performance and determine suitable hyperparameters.
2.The test set is used after training is completed to evaluate the model's generalization ability."

In [ ]:
from sklearn.model_selection import KFold

K = 10
kf = KFold(n_splits = K, random_state = 1, shuffle = True)
np.random.seed(1996)

## logistic regression


![](https://saedsayad.com/images/LogReg_1.png)

Logistic regression is a statistical method that is used for building machine learning models where the dependent variable is dichotomous: i.e. binary. Logistic regression is used to describe data and the relationship between one dependent variable and one or more independent variables. The independent variables can be nominal, ordinal, or of interval type.

The name “logistic regression” is derived from the concept of the logistic function that it uses. The logistic function is also known as the sigmoid function. The value of this logistic function lies between zero and one.


![](https://static.javatpoint.com/tutorial/machine-learning/images/linear-regression-vs-logistic-regression.png)

参考：https://zg104.github.io/Logistic_regression

**Why we use logistic regression, not linear regression? What are the disadvantages of linear regression for classification?**

- Linear regression can give us the values which are not between 0 and 1.

- Also, linear regression is sensitive to the outliers. However, the sigmoid function restrict the values between 0 and 1, which can be interpreted as the conditional probability of assigning the data to the particular class given the data parametrized by theta.

![](https://static.javatpoint.com/tutorial/machine-learning/images/linear-regression-vs-logistic-regression.png)

**What type of datasets is most suited for logistic regression?**

- Logistic regression likes overlapping data, instead of well separated data.
- Linear Discriminent Analysis will perform better for well separated data since the decision boundary is linear.

**Can you explain or interpret the hypothesis output of logistic regression?**

- We try to set a threshold to determine which class each data point should be assigned based on the conditional probability (I have clarified in Q1) derived from the sigmoid function.
- Typically, we set the threshold to be 0.5. However, it can be adjusted between 0 and 1 for personal specification, such as restriction on TPR (True Positive Rate).


**Why we define the sigmoid function, create a new version of cost function, and applied MLE to derive logistic regression?**

- Sigmoid function helps transform the linear esitimation into non-linear one.

- If we use the mean squared error as the cost function the same as linear regression, it is impossible to find the derivatives of the cost function with respect to theta, since the sigmoid function will make the cost function non-convex. So, we have to use gradient descent to minimize the cost function instead of computing the gradient by hand.

- You might wonder why the cost function of logistic regression is like this! That is beacuse we applied the MLE to maximize the probability to make the model the most plausible for all data points. You always minimize the loss function, which is just the negative form of the loglikelihood after MLE.

**How to deal with overfitting?**

- It can be pretty easy for every machine learning method to be overfitting. It is not a big deal!

- A regularization term is added to the cost function where the first part is loss function, and the second is the penalty term.

![](https://miro.medium.com/max/3232/1*vwhvjVQiEgLcssUPX6vxig.png)

**What are the disadvantage of logistic regression?**

- You should use k-fold cross validation to determine the highest polynomial of the features if the decision boundary is non-linear. It can be easy for this to overfit.

- Logistic regression is unstable when dealing with well separated datasets.

- Logistic regression requires relatively large datasets for training.

- Logistic regression is not that popular for multiclassification problems. Sigmoid function should be ungraded to Softmax function(You may hear about it if you know about Neural Networks).

<span id="5"></span>In order to get results between 0 and 1, a function, which is called **sigmoid**, is used to transform our hypothesis function. It is defined as
$$ $$
$$h_{\theta}(x) = g(\theta^{T} x)$$ 
$$ $$
where $h_{\theta}(x)$ is the hypothesis function, $x$ is a single record and 
$$ $$
$$g(z)=\dfrac{1}{1+e^{-z}}$$
$$ $$
By using $g(\theta^{T} x)$, we obtain the probablity and if $h_{\theta}(x) \geq 0.5$, we get $y=1$; if $h_{\theta}(x) < 0.5$, we get $y=0$. Further, when $z \geq 0$, $g(z) \geq 0.5$ is another detail. Thus, if the $\theta^{T} x \geq 0$, then $y=1$.
 
By the definition, I defined the below ***sigmoid*** function.<span id="5"></span>

We can't use the same cost function that we use for linear regression because the Logistic Function will cause the output to be wavy, causing many local optima. In other words, it will not be a convex function. That's why we need to define a different cost function for logistic regression. It is simply defined as
$$ $$
$$J(\theta) = \dfrac{1}{m} \sum^{m}_{i=1}Cost(h_{\theta}(x^{(i)}), y^{(i)})$$ 
$$ $$
where 
$$ $$
$$Cost(h_{\theta}(x^{(i)}), y^{(i)})=-y^{(i)} \; log(h_{\theta}(x^{(i)}))-(1-y^{(i)}) \; log(1-h_{\theta}(x^{(i)}))$$
$$ $$
As the sanity check, $J(\theta)$ can be plotted or printed as a function of the number of iterations to be sure that $J(\theta)$ is **decreasing on every iteration**, which shows that it is converging correctly. At this point, choice of $\alpha$ is important. If we select a high or small $\alpha$ value, we might have problem about the converging.<span id="6"></span>



In [ ]:
# prepare the data

# 1. 写出Sigmoid function

def sigmoid(z):
    return 1 / (1 + np.exp(-z))
    
# 2. loss function 
def loss(h, y):
    return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

## XGBoost

XGBoost is a supervised machine learning method for classification and regression and is used by the Train Using AutoML tool. XGBoost is short for extreme gradient boosting. This method is based on decision trees and improves on other methods such as random forest and gradient boost. 

To fit a training dataset using XGBoost, an initial prediction is made. Residuals are computed based on the predicted value and the observed values. A decision tree is created with the residuals using a similarity score for residuals. The similarity of the data in a leaf is calculated, as well as the gain in similarity in the subsequent split. The gains are compared to determine a feature and a threshold for a node. The output value for each leaf is also calculated using the residuals. For classification, the values are typically calculated using the log of odds and probabilities. The output of the tree becomes the new residual for the dataset, which is used to construct another tree. This process is repeated until the residuals stop reducing or for a specified number of times. Each subsequent tree learns from the previous trees and is not assigned equal weight, unlike how Random Forest works.

### Advantages of XGBoost:


1. Performance: XGBoost has a strong track record of producing high-quality results in various machine learning tasks, especially in Kaggle competitions, where it has been a popular choice for winning solutions.
2. Scalability: XGBoost is designed for efficient and scalable training of machine learning models, making it suitable for large datasets.
3. Customizability: XGBoost has a wide range of hyperparameters that can be adjusted to optimize performance, making it highly customizable.
4. Handling of Missing Values: XGBoost has built-in support for handling missing values, making it easy to work with real-world data that often has missing values.
5. Interpretability: Unlike some machine learning algorithms that can be difficult to interpret, XGBoost provides feature importances, allowing for a better understanding of which variables are most important in making predictions.

### Disadvantages of XGBoost:

1. Computational Complexity: XGBoost can be computationally intensive, especially when training large models, making it less suitable for resource-constrained systems.
2. Overfitting: XGBoost can be prone to overfitting, especially when trained on small datasets or when too many trees are used in the model.
3. Hyperparameter Tuning: XGBoost has many hyperparameters that can be adjusted, making it important to properly tune the parameters to optimize performance. However, finding the optimal set of parameters can be time-consuming and requires expertise.
4. Memory Requirements: XGBoost can be memory-intensive, especially when working with large datasets, making it less suitable for systems with limited memory resources.
The two main factors to choose XGBoost over other algorithms are:

### Reasons to Choose XGBoost

1. Execution Speed
2. Model Performance

![](https://dzone.com/storage/temp/13069535-xgboost-features.png)

### Hyperparameter tuning

- n_estimators: The number of trees or rounds. Adding more trees will be at the risk of overfitting. The reason is in the way that the boosted tree model is constructed, sequentially where each new tree attempts to model and correct for the errors made by the sequence of previous trees. Quickly, the model reaches a point of diminishing returns.

- max_depth: The maximum depth of a tree. It is also used to control overfitting as higher depth will allow model to learn relations very specific to a particular sample. Typically, it should be chosen from 3 to 10 and tuned using CV.

- objective: The loss function to be minimized. binary:logistic is for binary classification, which will return predicted probability (NOT CLASS).

- learning_rate: The convergence control parameter in gradient descent. It is intuitive that XGB will not reach its minimum if both n_estimaters and learning_rate are very small.

- subsample: The fraction of observations to be randomly chosen for each tree. Lower values make the algorithm more conservative and prevents overfitting, but too small values might lead to underfitting. So, be careful to choose and the typical values are between 0.5 and 1.

- min_child_weight: The minimum sum of weights all observations required in child. It is the minimum weight (or number of samples if all samples have a weight of 1) required in order to create a new node in the tree. A smaller min_child_weight allows the algorithm to create children that correspond to fewer samples, thus allowing for more complex trees, but again, more likely to overfit.

- colsample_bytree: The fraction of features to use. By default it is set to 1 meaning that we will use all features. But in order to avoid the number of highly correlated trees is getting too big, we would like to use a sample of all the features for training to avoid overfitting.

- scale_pos_weight: The parameter that controls the balance of positive and negative weights, useful for unbalanced classes. This dataset is unbalanced as we have seen, so we should be careful to tune it. The typical value to consider: sum(negative instances) / sum(positive instances).

- gamma: The minimum loss reduction required to make a split. A node is split only when the resulting split gives a positive reduction in the loss function. The larger gamma is, the more conservative (overfitting) the algorithm will be. The values can vary depending on the loss function and should be tuned.

- reg_alpha: L1 regularization term on weights. Increasing this value will make model more conservative.

- reg_lambda: L2 regularization term on weights. Increasing this value will make model more conservative. Normalised to number of training examples.

In [ ]:
# params = {
#         'min_child_weight': [1, 5, 10],
#         'gamma': [0.5, 1, 1.5, 2, 5, 10],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.6, 0.8, 1.0],
#         'max_depth': [3, 4, 5]
#         }

In [ ]:
# xgb = XGBClassifier(learning_rate=0.06, n_estimators=300, objective='binary:logistic',nthread=4)

In [ ]:
# from datetime import datetime
# def timer(start_time=None):
#     if not start_time:
#         start_time = datetime.now()
#         return start_time
#     elif start_time:
#         thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
#         tmin, tsec = divmod(temp_sec, 60)
#         print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))
        
        
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# from sklearn.model_selection import StratifiedKFold


# folds = 3
# param_comb = 5

# skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

# random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X,y), verbose=3, random_state=1001 )

# # Here we go
# start_time = timer(None)
# random_search.fit(X, y)
# timer(start_time) 

In [ ]:
# print('\n All results:')
# print(random_search.cv_results_)
# print('\n Best estimator:')
# print(random_search.best_estimator_)
# print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
# print(random_search.best_score_ * 2 - 1)
# print('\n Best hyperparameters:')
# print(random_search.best_params_)
# results = pd.DataFrame(random_search.cv_results_)
# results.to_csv('xgb-random-grid-search-results-01.csv', index=False)

### Optimization

In [ ]:
# 调参之后，较优的参数组合

from xgboost import XGBClassifier
MAX_ROUNDS = 400
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50  

model = XGBClassifier(    
                        n_estimators=MAX_ROUNDS,
                        max_depth=4,
                        objective="binary:logistic",
                        learning_rate=LEARNING_RATE, 
                        subsample=.8,
                        min_child_weight=6,
                        colsample_bytree=.8,
                        scale_pos_weight=1.6,
                        gamma=10,
                        reg_alpha=8,
                        reg_lambda=1.3,
                     )

In [ ]:
def XGB_gini(df_train,tar_enc = True,pca = False):
    
    '''
    df_train: 已处理的训练集数据
    tar_enc: 是否对类别型变量使用target encoding
    pca: 是否使用pca
    '''    
    
    y = df_train.target
    X = df_train.drop('target',axis=1)
    
    
    y_valid_pred = 0*y
    y_test_pred = 0
    
    
    from target_encoding import target_encode
    
    train = pd.concat([X,y],axis=1)
    for i, (train_index, test_index) in enumerate(kf.split(train)):

        # 分成训练集、验证集、测试集

        y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
        X_train, X_valid = X.iloc[train_index,:].copy(), X.iloc[test_index,:].copy()        
        X_test = final_test.copy()
        
        
        if pca == True:
            n_comp = 20
            print('\nPCA执行中...')
            pca = PCA(n_components=n_comp, svd_solver='full', random_state=1001)
            X_train = pd.DataFrame(pca.fit_transform(X_train))
            X_valid = pd.DataFrame(pca.transform(X_valid))
            X_test = pd.DataFrame(pca.transform(final_test.copy()))
        print( f"\n{i}折交叉验证： ")
        
        if pca == False:
            if tar_enc == True:
                f_cat = [f for f in X.columns if '_cat' in f and 'tar_enc' not in  f]
                for f in f_cat:
                    X_train[f + "_avg"], X_valid[f + "_avg"], X_test[f + "_avg"] = target_encode(
                                                                    trn_series=X_train[f],
                                                                    val_series=X_valid[f],
                                                                    tst_series=X_test[f],
                                                                    target=y_train,
                                                                    min_samples_leaf=100,
                                                                    smoothing=10,
                                                                    noise_level=0
                                                                    )

    #     from category_encoders.target_encoder import TargetEncoder
    #     tar_enc = TargetEncoder(cols = f_cat).fit(X_train,y_train)
    #     X_train = tar_enc.transform(X_train) # 转换训练集
    #     X_test = tar_enc.transform(X_test) # 转换测试集


            X_train.drop(f_cat,axis=1,inplace=True)
            X_valid.drop(f_cat,axis=1,inplace=True)
            X_test.drop(f_cat,axis=1,inplace=True)


        # 对于当前折，跑XGB
        if OPTIMIZE_ROUNDS:
            eval_set=[(X_valid,y_valid)]
            fit_model = model.fit( X_train, y_train, 
                                   eval_set=eval_set,
                                   eval_metric=gini_xgb,
                                   early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                                   verbose=False
                                 )
            print( "  Best N trees = ", model.best_ntree_limit )
            print( "  Best gini = ", model.best_score )
        else:
            fit_model = model.fit( X_train, y_train )

        # 生成验证集的预测结果
        pred = fit_model.predict_proba(X_valid)[:,1]
        print( "  normalized gini coefficent = ", eval_gini(y_valid, pred) )
        y_valid_pred.iloc[test_index] = pred

        # 累积计算测试集预测结果
        y_test_pred += fit_model.predict_proba(X_test)[:,1]

        del X_test, X_train, X_valid, y_train

    y_test_pred /= K  # 取各fold结果均值

    print( "\n整个训练集（合并）的normalized gini coefficent:" )
    print( "  final normalized gini coefficent = ", eval_gini(y, y_valid_pred) )
    
    return y_test_pred,eval_gini(y, y_valid_pred)

In [ ]:
%%time
y_test_pred, gini_score = XGB_gini(df_train=final_train,tar_enc=True)


0折交叉验证： 
  normalized gini coefficent =  0.2538658383235931

1折交叉验证： 
  normalized gini coefficent =  0.30728575577060424

2折交叉验证： 
  normalized gini coefficent =  0.28508943739615844

3折交叉验证： 
  normalized gini coefficent =  0.2829810430876034

4折交叉验证： 
  normalized gini coefficent =  0.29113416245470913

5折交叉验证： 
  normalized gini coefficent =  0.2940775662258974

6折交叉验证： 
  normalized gini coefficent =  0.2818130027819927

7折交叉验证： 
  normalized gini coefficent =  0.27666764206685135

8折交叉验证： 
  normalized gini coefficent =  0.2723987713495525

9折交叉验证： 
  normalized gini coefficent =  0.3070892136897676

整个训练集（合并）的normalized gini coefficent:
  final normalized gini coefficent =  0.28507826718380913
Wall time: 18min 55s


In [ ]:
submission = pd.DataFrame()
submission['id'] = final_test.index.values
submission['target'] = y_test_pred
submission.to_csv('xgb_submit.csv', float_format='%.6f', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=185fc705-b61f-4bb6-be30-c3c88dd0b19b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>